In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [131]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [132]:
generation_args = {
    "max_new_tokens": 512,
    "return_full_text": False,
    "do_sample": False,
}

In [22]:
def followup_que_gen_v0(question):
  messages=[{"role":"system",
             "content":"""You are a Socratic tutor. This is the student question:"""+ question + """. Please give me follow-up questions to guide
             the student to the correct answer, instead of providing direct answers. output each separately, in pure json, following this format:
             {
              "question":"%[question]"
             }
             Make sure each of the entry in its own, pure, json.
             Make sure generate maximum 10 questions."""}]
  print(messages)
  output = pipe(messages, **generation_args)

  socratic_response = output[0]['generated_text']

  return socratic_response

In [34]:
def followup_que_gen_v1(question):
    messages = [{"role": "system",
                 "content": f"""You are a Socratic tutor. This is the student question: '{question}'.
                 Please generate follow-up questions to guide the student to the correct answer.
                 Output the follow-up questions in **valid JSON format** like this:

                 {{
                     "questions": [
                         {{"question": "What is the first step in solving the equation?"}},
                         {{"question": "Can you identify the terms?"}},
                         ...
                     ]
                 }}

                 Ensure the output is **valid JSON**, not truncated, and contains up to 10 questions."""
                }]

    output = pipe(messages, **generation_args)
    socratic_response = output[0]['generated_text']

    return socratic_response

In [37]:
def followup_que_gen_v2(question):
    messages = [{"role": "system",
                 "content": f"""You are a Socratic tutor. This is the student question: '{question}'.
                 Please generate **an appropriate number of follow-up questions** to guide the student to the correct answer.
                 Only generate the number of questions that are necessary, up to a maximum of 10.
                 If fewer questions are sufficient, stop at the appropriate point.

                 Output the follow-up questions in **valid JSON format** like this:

                 {{
                     "questions": [
                         {{"question": "What is the first step in solving the equation?"}},
                         {{"question": "Can you identify the terms?"}},
                         ...
                     ]
                 }}

                 Ensure the output is **valid JSON**, not truncated, and contains an appropriate number of questions, based on the complexity of the student's question."""
                }]

    output = pipe(messages, **generation_args)
    socratic_response = output[0]['generated_text']

    return socratic_response


In [133]:
def followup_que_gen_v3(question):
    messages = [{"role": "system",
                 "content": f"""You are a Socratic tutor. This is the student question: '{question}'.
                 Please generate **an appropriate number of follow-up questions**, each with an expected answer, to guide the student to the correct answer.
                 Only generate the number of questions and answers that are necessary, up to a maximum of 10.
                 If fewer questions are sufficient, stop at the appropriate point.

                 Output the follow-up questions and their expected answers in **valid JSON format** like this:

                 {{
                     "questions_and_answers": [
                         {{"question": "What is the first step in solving the equation?",
                           "answer": "Isolate the variable by subtracting 3 from both sides."}},
                         {{"question": "Can you identify the terms?",
                           "answer": "The terms are 2x and 3 on the left side of the equation."}},
                         ...
                     ]
                 }}

                 Ensure the output is **valid JSON**, not truncated, and contains an appropriate number of questions and their corresponding expected answers, based on the complexity of the student's question."""
                }]

    output = pipe(messages, **generation_args)
    socratic_response = output[0]['generated_text']

    return socratic_response


In [47]:
student_question = "how to solve an 2x + 3 = 7 equation?"

In [ ]:
student_question = "If you drop a ball from a height of 10 meters, neglecting air resistance, how long will it take to reach the ground? And what will be its velocity just before hitting the ground?"

In [135]:
response = followup_que_gen_v3(student_question)
print(response)

 {
    "questions_and_answers": [
        {
            "question": "What is the formula to calculate the time it takes for an object to fall from a certain height under gravity, neglecting air resistance?",
            "answer": "The formula is t = sqrt(2h/g), where t is the time, h is the height, and g is the acceleration due to gravity (approximately 9.81 m/s^2 on Earth)."
        },
        {
            "question": "Using the formula t = sqrt(2h/g), can you calculate the time it takes for the ball to reach the ground from a height of 10 meters?",
            "answer": "t = sqrt(2*10/9.81) ≈ 1.43 seconds."
        },
        {
            "question": "What is the formula to calculate the final velocity of an object falling under gravity, neglecting air resistance?",
            "answer": "The formula is v = gt, where v is the final velocity, g is the acceleration due to gravity, and t is the time."
        },
        {
            "question": "Using the formula v = gt, can you calc

In [136]:
import json

input_text = response
# Extract only the JSON part (everything between the curly braces)
start_index = input_text.find('{')
end_index = input_text.rfind('}') + 1

json_text = input_text[start_index:end_index]

# Convert the extracted JSON string to a Python dictionary
try:
    json_object = json.loads(json_text)
    print(json_object)
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")


{'questions_and_answers': [{'question': 'What is the formula to calculate the time it takes for an object to fall from a certain height under gravity, neglecting air resistance?', 'answer': 'The formula is t = sqrt(2h/g), where t is the time, h is the height, and g is the acceleration due to gravity (approximately 9.81 m/s^2 on Earth).'}, {'question': 'Using the formula t = sqrt(2h/g), can you calculate the time it takes for the ball to reach the ground from a height of 10 meters?', 'answer': 't = sqrt(2*10/9.81) ≈ 1.43 seconds.'}, {'question': 'What is the formula to calculate the final velocity of an object falling under gravity, neglecting air resistance?', 'answer': 'The formula is v = gt, where v is the final velocity, g is the acceleration due to gravity, and t is the time.'}, {'question': 'Using the formula v = gt, can you calculate the velocity of the ball just before it hits the ground after falling for approximately 1.43 seconds?', 'answer': 'v = 9.81 * 1.43 ≈ 14.02 m/s.'}]}


In [137]:
generation_args = {
    "max_new_tokens": 32,
    "return_full_text": False,
    "do_sample": False,
}

In [138]:
def check_answer_prompt(question, expected_answer, user_answer):
    prompt = [{"role":"system", "content":f"""
    You are an AI that checks answers for correctness.

    - The question is: "{question}"
    - The expected answer is: "{expected_answer}"
    - The user's answer is: "{user_answer}"

    Based on the user's answer compared to the expected answer, determine if the user's answer is **correct** or **incorrect**.

    Output only in valid JSON format like this:

    {{
        "result": "correct"  # or "incorrect"
    }}
    """}]

    output = pipe(prompt, **generation_args)
    response = output[0]['generated_text']

    return response



In [139]:
question = "What is the formula to calculate the time it takes for an object to fall from a certain height under gravity, neglecting air resistance?",
expected_answer = "The formula is t = sqrt(2h/g), where t is the time, h is the height, and g is the acceleration due to gravity (approximately 9.81 m/s^2 on Earth)."
user_answer = "don't know"

In [54]:
question = "What is the first step in solving the equation 2x + 3 = 7?"
expected_answer = "Subtract 3 from both sides to isolate the term with the variable."
user_answer = "add 7 to 3"

response = check_answer_prompt(question, expected_answer, user_answer)
print(response)

 {
    "result": "incorrect"
}

The correct first step in solving the equation 2x + 3 = 7 is


In [55]:
import json

input_text = response
# Extract only the JSON part (everything between the curly braces)
start_index = input_text.find('{')
end_index = input_text.rfind('}') + 1

json_text = input_text[start_index:end_index]

# Convert the extracted JSON string to a Python dictionary
try:
    json_object = json.loads(json_text)
    print(json_object)
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")

{'result': 'incorrect'}


In [140]:
generation_args = {
    "max_new_tokens": 64,
    "return_full_text": False,
    "do_sample": False,
}

In [141]:
def ask_question_in_tutor_tone(question):
    prompt = [{"role":"system", "content":f"""
    You are a helpful tutor. Ask the following question in a concise, teacher-like tone, encouraging them to think critically but keeping the question short:

    The question is: "{question}"

    Make sure the phrasing is supportive, but the question is brief and to the point.
    """}]

    output = pipe(prompt, **generation_args)
    response = output[0]['generated_text']

    return response


In [142]:
question = "What is the formula to calculate the time it takes for an object to fall from a certain height under gravity, neglecting air resistance?"

response = ask_question_in_tutor_tone(question)
print(response)

 Can you derive the formula for the time it takes for an object to fall from a given height, assuming no air resistance, using the acceleration due to gravity?


In [109]:
question="What is the next step after simplifying the equation to 2x = 4?"
correct_answer="Divide both sides by 2 to solve for x."
student_answer="divide both side by 3"
max_attempts=2

In [148]:
generation_args = {
    "max_new_tokens": 512,
    "return_full_text": False,
    "do_sample": False,
}

In [144]:
# Initialize conversation history
conversation_history = []

In [145]:
  # System prompt for navigating the student to the correct answer
prompt = "You are a helpful tutor who guide them to the correct solution by provide supportive hints to help them to get the correct answer. Use a friendly, encouraging tone in your responses."

conversation_history.append({"role": "system", "content": prompt})

In [149]:


# Function to check if the answer is correct and guide the student
def tutor_with_guidance_v0(question, correct_answer, student_answer):
    prompt = ""

    conversation_history.append({"role":"assistant", "content": question})

    # conversation_history.append({"role": "user", "content": f"Student's answer: {student_answer}"})


    int_response = check_answer_prompt(question, correct_answer, student_answer)
    input_text = int_response

    start_index = input_text.find('{')
    end_index = input_text.rfind('}') + 1

    int_json_text = input_text[start_index:end_index]

    try:
        json_object = json.loads(int_json_text)
        print(json_object)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")

    # System prompt for navigating the student to the correct answer
    prompt = f"""
    The correct answer is: '{correct_answer}'. The student's answer is: '{student_answer} which is incorrect'.

    Based on the student's answer and its correctness, provide steps to get the correct answer.

    Use a friendly, encouraging tone in your responses. Make sure to keep your response in short.
    """

    conversation_history.append({"role": "user", "content": prompt})

    # Generate the output based on the current conversation history
    output = pipe(conversation_history, **generation_args)
    socratic_response = output[0]['generated_text']

    conversation_history.append({"role": "assistant", "content": socratic_response})
    print(f"Tutor: {socratic_response}")





In [117]:
print(conversation_history)

[{'role': 'system', 'content': 'You are a helpful tutor who guide them to the correct solution by provide supportive hints to help them to get the correct answer. Use a friendly, encouraging tone in your responses.'}]


In [112]:
# Initialize conversation history
conversation_history = []

# Function to check if the answer is correct and guide the student
def tutor_with_guidance(question, correct_answer, max_attempts=4):
    attempts = 0
    prompt = ""

    while attempts < max_attempts:
        if attempts > 0:
            # Ask for the student's new answer after the first attempt
            student_answer = input("Your new answer: ")
        else:
            # First student answer
            student_answer = input("Your answer: ")

        # Add the student's answer to the conversation history
        conversation_history.append({"role": "user", "content": f"Student's answer: {student_answer}"})

        # Check if the answer is correct using check_answer_prompt
        int_response = check_answer_prompt(question, correct_answer, student_answer)
        input_text = int_response

        # Extract only the JSON part (everything between the curly braces)
        start_index = input_text.find('{')
        end_index = input_text.rfind('}') + 1
        int_json_text = input_text[start_index:end_index]

        # Convert the extracted JSON string to a Python dictionary
        try:
            json_object = json.loads(int_json_text)
            print(f"Check result: {json_object['result']}")
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")
            break  # Exit if we cannot parse the response properly

        # Update conversation based on correctness of the answer
        prompt = f"""This is the question: '{question}'.
        The correct answer is: '{correct_answer}'. The student's answer is: '{student_answer}', which is {json_object['result']}.

        Based on the student's answer and its correctness, help guide him to the correct solution by provide supportive hints to
        help them get to the correct answer.

        Use a friendly, encouraging tone in your responses. Make sure to keep your response in brief.
        """

        conversation_history.append({"role": "system", "content": prompt})

        # Generate the output based on the current conversation history
        output = pipe(conversation_history, **generation_args)
        socratic_response = output[0]['generated_text']
        print(f"Tutor: {socratic_response}")

        attempts += 1

        # Check if the student's answer matches the correct answer
        if json_object['result'] == 'correct':
            print("Tutor: Correct! Well done!")
            break

        # If the student fails after max_attempts, provide the correct answer
        if attempts == max_attempts:
            print(f"Tutor: You've given {max_attempts} attempts. The correct answer is: {correct_answer}")
            conversation_history.append({"role": "assistant", "content": f"The correct answer is: {correct_answer}"})
            break

In [150]:
# Run the function
tutor_with_guidance_v0(question, correct_answer, student_answer)

{'result': 'incorrect'}
Tutor:  No worries, let's clarify this together!

The student's mention of dividing by 2 or 3 seems to be a bit off-track. Here's a simplified way to find the correct formula for the time (t) it takes for an object to fall from a certain height (h) under gravity (g):

1. Start with the equation for an object in free fall: h = 0.5 * g * t^2

2. To solve for t, we need to isolate t^2. We do this by dividing both sides by (0.5 * g):

   t^2 = h / (0.5 * g)

3. Now, to find t, we take the square root of both sides:

   t = sqrt(h / (0.5 * g))

So, the correct formula is t = sqrt(h / (0.5 * g)). Remember, we're not dividing by 2 or 3 here. We're dividing by 0.5 * g to isolate t^2, and then taking the square root to solve for t. Keep practicing, and you'll get the hang of it!


In [119]:
print(conversation_history)

[{'role': 'system', 'content': 'You are a helpful tutor who guide them to the correct solution by provide supportive hints to help them to get the correct answer. Use a friendly, encouraging tone in your responses.'}, {'role': 'assistant', 'content': 'What is the next step after simplifying the equation to 2x = 4?'}, {'role': 'user', 'content': "\n    The correct answer is: 'Divide both sides by 2 to solve for x.'. The student's answer is: 'divide both side by 3 which is incorrect'.\n\n    Based on the student's answer and its correctness, provide supportive hints to get the correct answer.\n\n    Use a friendly, encouraging tone in your responses.\n    "}, {'role': 'assistant', 'content': " Hey there! You're on the right track by thinking about dividing both sides of the equation to solve for x. However, it looks like there's a small mix-up with the numbers. Let's take a closer look together:\n\nYou have the equation 2x = 4. To find the value of x, we want to isolate it on one side of